In [24]:
import numpy as np, pandas as pd, xarray as xr
import matplotlib
%matplotlib qt

In [25]:

all = np.load("/media/filer2/T4/Julien/Luisa/Rat26_20240416_newtask21_data/Lick.npy")
fs = pd.read_csv("/media/filer2/T4/Julien/Luisa/Rat26_20240416_newtask21.tsv", sep="\t", index_col=0).loc["Lick", "fs"]
(fs, arr.shape)

(20000.0, (200001,))

In [26]:
arr = xr.DataArray(all, dims="t")
t = xr.DataArray(np.arange(all.size)/fs, dims="t")
arr["t"] = t
# arr=arr.sel(t=slice(40 , 50))
t = arr["t"]
arr

<xarray.DataArray (t: 51102388)> Size: 409MB
array([-0.0491333 , -0.05661011, -0.05935669, ...,  0.12710571,
        0.12374878,  0.12649536])
Coordinates:
  * t        (t) float64 409MB 0.0 5e-05 0.0001 ... 2.555e+03 2.555e+03

In [33]:
arr.plot()

In [28]:
binary_licks: xr.DataArray = np.abs(arr) < np.abs(arr).max()/20
print(np.abs(arr).max()/20)
binary_licks

<xarray.DataArray ()> Size: 8B
array(0.01193237)


<xarray.DataArray (t: 51102388)> Size: 51MB
array([False, False, False, ..., False, False, False])
Coordinates:
  * t        (t) float64 409MB 0.0 5e-05 0.0001 ... 2.555e+03 2.555e+03

In [29]:
is_rising = (binary_licks & ~binary_licks.shift(t=1, fill_value=False))
is_falling = ~binary_licks & binary_licks.shift(t=1, fill_value=False)
xr.Dataset(dict(is_rising=is_rising, is_falling=is_falling, count_r = is_rising.sum(), count_f = is_falling.sum()))



<xarray.Dataset> Size: 511MB
Dimensions:     (t: 51102388)
Coordinates:
  * t           (t) float64 409MB 0.0 5e-05 0.0001 ... 2.555e+03 2.555e+03
Data variables:
    is_rising   (t) bool 51MB False False False False ... False False False
    is_falling  (t) bool 51MB False False False False ... False False False
    count_r     int64 8B 756720
    count_f     int64 8B 756720

In [30]:
rising_events = t.where(is_rising, drop=True).expand_dims(rise=[True]).rename(t="events").reset_index("events", drop=True)
falling_events = t.where(is_falling, drop=True).expand_dims(rise=[False]).rename(t="events").reset_index("events", drop=True)
if falling_events.isel(events=0) < rising_events.isel(events=0):
    falling_events = falling_events.isel(events=slice(1, None))
changes = xr.concat([rising_events, falling_events], dim="rise").rename("ev_times")
changes

<xarray.DataArray 'ev_times' (rise: 2, events: 756720)> Size: 12MB
array([[8.40000000e-03, 9.50000000e-03, 1.85000000e-02, ...,
        2.55510540e+03, 2.55511480e+03, 2.55511490e+03],
       [9.45000000e-03, 9.55000000e-03, 1.86500000e-02, ...,
        2.55510610e+03, 2.55511485e+03, 2.55511605e+03]])
Coordinates:
  * rise     (rise) bool 2B True False
Dimensions without coordinates: events

In [35]:
if not ((changes.sel(rise=False) - changes.sel(rise=True))< 100).all():
    print("Error")
if not ((changes.sel(rise=False) - changes.sel(rise=True))> 0).all():
    print("Error")


In [34]:
licks = changes.where(changes.sel(rise=False) - changes.sel(rise=True) > 0.025, drop=True)
licks

<xarray.DataArray 'ev_times' (rise: 2, events: 1300)> Size: 21kB
array([[  14.23265,   14.6152 ,   14.8099 , ..., 2422.16815, 2542.483  ,
        2553.5321 ],
       [  14.2937 ,   14.67785,   14.8746 , ..., 2422.22985, 2542.5138 ,
        2553.57125]])
Coordinates:
  * rise     (rise) bool 2B True False
Dimensions without coordinates: events